# Video Processing for BT Lab
This notebook extracts metadata from video files 

<br>

![UofC logo](./pictures/uofc_logo-black.jpg)

In [46]:
#import libraries
import os
import cv2
import json
import json
import datetime
import pandas as pd

In [ ]:
repo_dir = os.getcwd()
json_dir = repo_dir + "/record/JSON"

In [ ]:
def get_video_properties(mp4_file: str) -> list:
    try:
        video_properties = []
        video = cv2.VideoCapture(mp4_file)
        old_fps = video.get(cv2.CAP_PROP_FPS)
        number_of_frames = video.get(cv2.CAP_PROP_FRAME_COUNT)
        length = number_of_frames / old_fps
        video_time = datetime.timedelta(seconds=length)
        video_properties.append(number_of_frames)
        video_properties.append(old_fps)
        video_properties.append(length)
        video_properties.append(video_time)
    except Exception as e:
        print(f"{type(e)} - from {mp4_file}")
    
    return video_properties

In [ ]:
def add_video_properties(metadata:dict) -> dict:
    new_patient_info = {}

    for patient_id, patient_info in metadata.items():
        try:
            video_path = patient_info["local path"]
            frames, fps, length_in_seconds, length_in_datetime = get_video_properties(video_path)
            patient_info["frames"] = frames
            patient_info["old fps"] = fps
            patient_info["length"] = length_in_seconds
            patient_info["length (datetime)"] = length_in_datetime
            new_patient_info[patient_id] = patient_info
        except Exception as e:
            print(f"{type(e)} - {patient_info['ID']}")

    return new_patient_info 

In [ ]:
def load_json(json_dir:str, filename:str) -> dict:
    full_path = json_dir + "/" + filename

    with open(full_path, "r") as json_data:
        data = json.load(json_data)

    return(data)

In [63]:
def export_to_excel(patient_info:dict, filename: str):
    rgb_df = pd.DataFrame.from_dict(patient_info).transpose()
    rgb_df = rgb_df.drop(columns="local path")
    new_df = rgb_df.set_index("ID")
    new_df.to_excel(filename)
    return new_df

## Get video metadata and export to xlsx file (stage 2.1)

In [ ]:
""" load JSON files """

metadata_rgb = load_json(json_dir, "rgb.json")
metadata_thermal = load_json(json_dir, "thermal.json")

In [ ]:
""" extract metadata from video (rgb) """

complete_metadata_rgb = add_video_properties(metadata_rgb)
complete_metadata_rgb

In [64]:
""" patient info to pandas DataFrame"""
rgb_xlsx = os.getcwd() + "/record/xlsx/rgb.xlsx"
rgb_df = export_to_excel(complete_metadata_rgb, rgb_xlsx)

In [69]:
rgb_df

,distance,blanket,breathing,filename,frames,length,old fps,length (datetime)
ID,,,,,,,,
Arun,2 Meters,With Blankets,Hold Breath,Arun2.mp4,2146.0,73.054497,29.375331,0 days 00:01:13.054497
Arun,2 Meters,With Blankets,Relaxed,Arun2.mp4,143.0,7.703175,18.563774,0 days 00:00:07.703175
Arun,2 Meters,Without Blankets,Hold Breath,short.mp4,1790.0,36.113806,49.565532,0 days 00:00:36.113806
Arun,2 Meters,Without Blankets,Relaxed,relax 2meter short.mp4,3872.0,72.04854,53.741547,0 days 00:01:12.048540
Arun,3 Meters,With Blankets,Hold Breath,Arun3.mp4,1340.0,37.059608,36.157965,0 days 00:00:37.059608
...,...,...,...,...,...,...,...,...
Sara,2 Meters,Without Blankets,Relaxed,Sara Relaxed.mp4,1497.0,24.849016,60.243834,0 days 00:00:24.849016
Sara,3 Meters,With Blankets,Hold Breath,Sara HB.mp4,1252.0,20.772879,60.270895,0 days 00:00:20.772879
Sara,3 Meters,With Blankets,Relaxed,Sara Relaxed.mp4,1222.0,20.266021,60.297974,0 days 00:00:20.266021


In [68]:
rgb_df.describe(include="all")

,distance,blanket,breathing,filename,frames,length,old fps,length (datetime)
count,135,135,135,135,135.0,135.000000,135.000000,135
unique,4,4,5,55,130.0,135.000000,135.000000,NaN
top,2 Meters,Without Blankets,Relaxed,Farooq.mp4,2808.0,73.054497,29.375331,NaN
freq,68,65,66,5,2.0,1.000000,1.000000,NaN
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0 days 00:00:37.495670925
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0 days 00:00:18.941263001
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0 days 00:00:04.113682
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0 days 00:00:26.817246500
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0 days 00:00:32.200929
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0 days 00:00:47.044679
